In [1]:
import pandas as pd
import csv

In [8]:
# Importing files (just once because it takes LOOOOOOOOONG)

# Results (not that useful)
results_june_2024 = pd.read_excel('../../20_data/regelleistung/RESULT_OVERVIEW_ENERGY_MARKET_aFRR_2024-06-01_2024-06-30.xlsx')

# Only goes up to 2024-06-11
#bids_june_2024_import = pd.read_excel('../../20_data/regelleistung/RESULT_LIST_ANONYM_ENERGY_MARKET_aFRR_2024-06-01_2024-06-30.xlsx')
#bids_june_2024_import.to_csv('../../20_data/regelleistung/RESULT_LIST_ANONYM_ENERGY_MARKET_aFRR_2024-06-01_2024-06-30.csv')
bids_june_2024_import = pd.read_csv('../../20_data/regelleistung/RESULT_LIST_ANONYM_ENERGY_MARKET_aFRR_2024-06-01_2024-06-30.csv',low_memory=False)
bids_june_2024_import.drop(columns=['Unnamed: 0'], inplace=True)

# Whole month
activated_all_time_import = pd.read_csv('../../20_data/netztransparenz/Aktivierte aFRR qualitaetsgesichert [2024-09-26 15-50-49].csv', delimiter = ';')

/Users/nicolas/miniforge3/envs/analysis/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:241: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [9]:
# Preparing activation data

activated_all_time = activated_all_time_import.copy()
activated_all_time.Datum = pd.to_datetime(activated_all_time.Datum, format = "%d.%m.%Y")

#Only 01-11 June 2024
activated_june_2024 = activated_all_time[(activated_all_time.Datum.dt.month == 6) & (activated_all_time.Datum.dt.year == 2024) & (activated_all_time.Datum.dt.day <= 11)]

# Remove TSO specific columns, add product names, replace commas with dots, rename columns
activated_june_2024 = activated_june_2024.drop(columns = ['50Hertz (Positiv)','Amprion (Positiv)', 'TenneT TSO (Positiv)', 'TransnetBW (Positiv)','50Hertz (Negativ)', 'Amprion (Negativ)','TenneT TSO (Negativ)', 'TransnetBW (Negativ)'])
activated_june_2024['product_time'] = activated_june_2024.von.apply(lambda x: x.split(':')).apply(lambda x: int((4*int(x[0])) +(int(x[1])/15)+1)).apply(lambda x: (3-len(str(x)))*'0' + str(x))
activated_june_2024['Deutschland (Positiv)'] = activated_june_2024['Deutschland (Positiv)'].apply(lambda x: x.replace(',','.'))
activated_june_2024['Deutschland (Negativ)'] = activated_june_2024['Deutschland (Negativ)'].apply(lambda x: x.replace(',','.'))
activated_june_2024.rename(columns = {'Deutschland (Positiv)':'POS', 'Deutschland (Negativ)':'NEG'}, inplace = True)

#Separating Neg and Pos products into separate rows
activated_june_2024['time_prod_NEG'] = 'NEG_' + activated_june_2024['product_time'] 
activated_june_2024['time_prod_POS'] = 'POS_' + activated_june_2024['product_time']
NEG = activated_june_2024.rename(columns={'time_prod_NEG':'prod'}).set_index(['Datum','prod']).drop(columns = ['POS']).rename(columns={'NEG':'ACTIVATED'}).drop(columns=['time_prod_POS'])
POS = activated_june_2024.rename(columns={'time_prod_POS':'prod'}).set_index(['Datum','prod']).drop(columns = ['NEG']).rename(columns={'POS':'ACTIVATED'}).drop(columns=['time_prod_NEG'])
combined_activated = pd.concat([NEG,POS])

In [47]:
# Preparing bidding data

bids_june_2024 = bids_june_2024_import.copy()
bids_june_2024['DELIVERY_DATE'] = pd.to_datetime(bids_june_2024.DELIVERY_DATE,format = "%Y-%m-%d")

# Translate payment direction into sign for price
bids_june_2024['ENERGY_PRICE_[EUR/MWh]'] = bids_june_2024['ENERGY_PRICE_[EUR/MWh]']*(((bids_june_2024['ENERGY_PRICE_PAYMENT_DIRECTION'] == 'PROVIDER_TO_GRID') * -1) + ((bids_june_2024['ENERGY_PRICE_PAYMENT_DIRECTION'] == 'GRID_TO_PROVIDER') * 1))

# Sort by Dalivery_date, product then energy price (check order)
bids_sorted = bids_june_2024.sort_values(['DELIVERY_DATE','PRODUCT','ENERGY_PRICE_[EUR/MWh]'])

# Cumulative sums of offered capacity
bids_sorted['sum_cap'] = bids_sorted.groupby(['DELIVERY_DATE','PRODUCT'])['OFFERED_CAPACITY_[MW]'].cumsum()

bids_activated = bids_sorted.set_index(['DELIVERY_DATE','PRODUCT'])


In [52]:
# Aligning activation and bidding data

# Join both
bids_activated['act'] = combined_activated['ACTIVATED']

# Keep only activated energy bids (exclude partially activated)
bids_activated = bids_activated[bids_activated['sum_cap'] < bids_activated['act'].astype(float)]

#bids_activated[(bids_activated.index.get_level_values('DELIVERY_DATE') == pd.to_datetime('2024-06-05')) & (bids_activated.index.get_level_values('PRODUCT') == 'POS_048')]# == [pd.to_datetime('2024-06-01'), 'NEG_001']]
# Daily activation cost (assuming full activation for 15min) 
#((bids_activated['ENERGY_PRICE_[EUR/MWh]'].groupby('DELIVERY_DATE').sum())/4).astype(int)
((bids_activated['ENERGY_PRICE_[EUR/MWh]'].groupby('PRODUCT').sum())/4).astype(int)

PRODUCT
NEG_001     -353
NEG_002     -310
NEG_003     -250
NEG_004     -361
NEG_005     -492
           ...  
POS_092      971
POS_093    11843
POS_094     5552
POS_095     4790
POS_096     2428
Name: ENERGY_PRICE_[EUR/MWh], Length: 192, dtype: int64